In [28]:
# spell-error 没有标写错是哪个单词的概率，视为一样的概率
# vocab 是词库
# testdata是测试集

"""
构建词库
生成候选模型
构建语言模型
"""

# 1 构建词库
vocab = set([line.rstrip() for line in open('vocab.txt')])
[w for w in vocab if w=='apprle']

[]

In [32]:
# 需要生成所有候选集合
def generate_candidate(word):
    """
    word:给定的输入(错误的输入)
    返回所有(valid)候选集合
    """
    # 生成编辑距离为1的单词
    # 1.insert 2.delete 3.replace
    # appl : replace:bppl, cppl, aapl
    
    #假设使用26个字符
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
#     print(splits)
    
    # insert
    inserts = [L+c+R for L,R in splits for c in letters]
#     print(inserts)
    # delete 
    delete = [L+R[1:] for L,R in splits if R]
#     print(delete)
    # replace
    replace =[L+c+R[1:] for L,R in splits if R for c in letters if c!=R[0]]
#     print(replace)
    
    candidates = set(inserts+delete+replace+[word])
    
    # 过滤掉不存在于词典库里的单词
    return [w for w in candidates if w in vocab]
    
generate_candidate('apple')

['apples', 'ample', 'apply', 'apple']

In [6]:
# 语言模型
from nltk.corpus import reuters #dataset
# 读取语料库
categories = reuters.categories()
corpus = reuters.sents(categories=categories)

In [5]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to /usr/local/lib/nltk_data...


True

In [10]:
# 构建语言模型， bigram
term_count = {}
bigram_count = {}
for doc in corpus: # 每个doc都是由几句话组成的文章
    doc = ['<s>'] + doc # 让开头单词也有bigram的情况 P(I|<s>)
    for i in range(0, len(doc)-1):
        # bigram : [i,i+1]
        term = doc[i]
        bigram = doc[i:i+2]
        
        if term in term_count:
            term_count[term]+=1
        else:
            term_count[term]=1
        
        bigram = ' '.join(bigram)
        if bigram in bigram_count:
            bigram_count[bigram]+=1
        else:
            bigram_count[bigram]=1
print(term_count)
# sklearn有现成的语言模型包

{'<s>': 54716, 'ASIAN': 12, 'EXPORTERS': 46, 'FEAR': 2, 'DAMAGE': 13, 'FROM': 208, 'U': 6388, '.': 45900, 'S': 6382, '.-': 167, 'JAPAN': 295, 'RIFT': 1, 'Mounting': 1, 'trade': 2271, 'friction': 28, 'between': 1053, 'the': 58251, 'And': 227, 'Japan': 1594, 'has': 4679, 'raised': 334, 'fears': 65, 'among': 229, 'many': 297, 'of': 35979, 'Asia': 71, "'": 11272, 's': 9298, 'exporting': 51, 'nations': 399, 'that': 7377, 'row': 49, 'could': 1431, 'inflict': 1, 'far': 275, '-': 13705, 'reaching': 37, 'economic': 784, 'damage': 128, ',': 72359, 'businessmen': 56, 'and': 25043, 'officials': 836, 'said': 25221, 'They': 562, 'told': 1424, 'Reuter': 31, 'correspondents': 7, 'in': 26478, 'Asian': 53, 'capitals': 7, 'a': 23492, 'Move': 1, 'against': 1428, 'might': 399, 'boost': 203, 'protectionist': 86, 'sentiment': 61, 'lead': 261, 'to': 34035, 'curbs': 36, 'on': 8556, 'American': 817, 'imports': 776, 'their': 1345, 'products': 703, 'But': 1157, 'some': 1198, 'exporters': 195, 'while': 812, 'confl

In [33]:
# 用户打错的概率统计 - channel probability 
channel_prob = {}

for line in open('spell-errors.txt'):
    items = line.split(":")
    correct = items[0].strip()
    mistakes = [item.strip() for item in items[1].strip().split(",")]
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0/len(mistakes)
channel_prob


{'raining': {'rainning': 0.5, 'raning': 0.5},
 'writings': {'writtings': 1.0},
 'disparagingly': {'disparingly': 1.0},
 'yellow': {'yello': 1.0},
 'four': {'forer': 0.2,
  'fours': 0.2,
  'fuore': 0.2,
  'fore*5': 0.2,
  'for*4': 0.2},
 'woods': {'woodes': 1.0},
 'hanging': {'haing': 1.0},
 'aggression': {'agression': 1.0},
 'looking': {'loking': 0.1,
  'begining': 0.1,
  'luing': 0.1,
  'look*2': 0.1,
  'locking': 0.1,
  'lucking': 0.1,
  'louk': 0.1,
  'looing': 0.1,
  'lookin': 0.1,
  'liking': 0.1},
 'eligible': {'eligble': 0.3333333333333333,
  'elegable': 0.3333333333333333,
  'eligable': 0.3333333333333333},
 'electricity': {'electrisity': 0.3333333333333333,
  'electricty*2': 0.3333333333333333,
  'electrizity': 0.3333333333333333},
 'scold': {'schold': 0.5, 'skold': 0.5},
 'adaptable': {'adabtable': 1.0},
 'caned': {'canned': 0.5, 'cained': 0.5},
 'immature': {'imature': 1.0},
 "shouldn't": {'shoudln': 0.5, 'shouldnt': 0.5},
 'swivel': {'swival': 1.0},
 'appropriation': {'apro

In [39]:
import numpy as np
V = len(term_count.keys())
file = open("testdata.txt",'r')
for line in file:
    items = line.rstrip().split('\t')
    line = items[2].split()
    # line = ["I","like"]
    for word in line:
        if word not in vocab:
            # 需要替换word成正确的单词
            # 1 生成所有的候选集合
            candidates = generate_candidate(word)
            # if candidate =[] 可以多生成一个candidats 编辑距离不超过2的
            if len(candidates) < 1:
                continue # 这里粗暴解决 TODO
                
            probs = []
            # 对于每个candidate计算他的score
            # score = p(correct)*p(mistake|correct)
            #       = log p(correct) + log(p(mistake|correct))
            # 返回score 最大的candidate
            for candi in candidates:
                prob = 0
                # a. 计算channel probability
                if candi in channel_prob and word in channel_prob[candi]:
                    prob += np.log(channel_prob[candi][word])
                else:
                    prob += np.log(0.0001)
                # b. 计算语言模型的概率
                idx = items[2].index(word)+1
                if items[2][idx-1] in bigram_count and candi in bigram_count[items[2][idx-1]]:
                    prob += np.log((bigram_count[items[2][idx-1]][candi] + 1.0 ))/ (term_count[bigram_count[items[2][idx-1]]] + V)
                # TODO：考虑了前面的bigram 也要考虑当前word和post_word后面词的 概率
                # prob+=np.log(bigram(word,post_word))
                else:
                    prob += np.log(1.0/V)
                probs.append(prob)
            max_idx = probs.index(max(probs))
            print(word, candidates[max_idx])

protectionst protectionist
products. products
long-run, long-run
gain. gain
17, 17e
retaiation retaliation
cost. cost
busines, business
ltMC.T. ltMC.T
U.S., U.S.
Murtha, Murtha
worried. worried
seriousnyss seriousness
aganst against
us, us
named. named
year, yearn
sewll sell
dlrs, dlrs
world's worlds
largest. largest
markets, markets
importsi imports
Products, Products
Retaliation, Retaliation
Group. Group
Korea's Koreans
Korea, Korea
Japan. Japan
Koreva Korea
U.S., U.S.
1985. 1985
Malaysia, Malaysian
Japn Japan
Kong, Kong
view. view
advantagne advantage
imports. imports
view, view
Lawrenc Lawrence
Mills, Mills
Industry. Industry
imports, imports
sources. sources
disadxantage disadvantage
trade, trader
said. said
market, markets
Friday. Fridays
said. said
cenntred centred
beef, beefy
country. country
trad trade
continue. continued
Liberala Liberals
economy. economy
inoclude include
year. yearn
program. program
Representetive Representative
Kuroda, Kuroda
MITI, MITI
dispute. disputes
to